In [ ]:
pip install streamlit google-generativeai chromadb langchain_google_genai  faiss-gpu PyPDF2 langchain python-dotenv pdfplumber


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("/content/drive/MyDrive/Colab Notebooks/api.env")
print(config)
import os
os.environ["GOOGLE_API_KEY"]="YOUR GOOGLE API"

In [ ]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
#from IPython.display import display, HTML
import pdfplumber
import textwrap
# Configuration
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Functions for PDF processing and text extraction
def get_pdf_text(file_paths):
    text = ""
    for file_path in file_paths:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() if page.extract_text() else " "
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

# Conversational chain setup
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    return load_qa_chain(model, chain_type="stuff", prompt=prompt)

# Question handling
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    print(response)
    return response

# Main interaction loop in Jupyter
import os
from dotenv import load_dotenv
from IPython.display import display, HTML

# Assuming all necessary functions like get_pdf_text, get_text_chunks, get_vector_store, and user_input are defined elsewhere

def main():
    load_dotenv()
    pdf_path = [
        '/content/drive/MyDrive/Colab Notebooks/1) JP 5-0 Joint Planning (01DEC20).pdf',
        '/content/drive/MyDrive/Colab Notebooks/2) JP 3-05 Joint Doctrine for Special Operations.pdf',
        '/content/drive/MyDrive/Colab Notebooks/3) JP 3-04 Information in Joint Operations.pdf',
        '/content/drive/MyDrive/Colab Notebooks/4) CJCSI 3110.05G MISO Supplement to Joint Strategic Campaign Plan.pdf',
        '/content/drive/MyDrive/Colab Notebooks/5) CJCSM 3105.01A Joint Risk Analysis Methodology.pdf',
        '/content/drive/MyDrive/Colab Notebooks/6) Defense Primer_What is Irregular Warfare.pdf',
        '/content/drive/MyDrive/Colab Notebooks/7) RAND_Intelligence Support for Operations in the Information Environment.pdf',
        '/content/drive/MyDrive/Colab Notebooks/jp1.pdf'
        # Add other PDFs as needed
    ]
    print("Loading PDFs and processing text...")
    raw_text = get_pdf_text(pdf_path)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)
    print("Setup complete. You can now start asking questions.")

    while True:
        user_question = input("\nAsk a Question from the PDF Files (type 'exit' to quit): ").strip()
        if user_question.lower() == 'exit':
            print("\nExiting the Q&A session. Thank you!")
            break

        try:
            response = user_input(user_question)
            # Using plain print instead of HTML display
            wrapped_question = textwrap.fill(user_question, width=80)
            wrapped_answer = textwrap.fill(response['output_text'],width=80)
            print("\n" + "="*80)
            print(f"Question:\n{wrapped_question}\n")
            print(f"Answer:\n{wrapped_answer}")
            print("="*80+ "\n")
        except Exception as e:
            print(f"\nError processing the question: {e}\n")

if __name__ == "__main__":
    main()

Loading PDFs and processing text...
Setup complete. You can now start asking questions.

Ask a Question from the PDF Files (type 'exit' to quit): what is OPCON


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'output_text': 'OPCON is the command authority that may be exercised by commanders at any echelon at or below the level of CCMD and may be delegated within the command.'}

Question:
what is OPCON

Answer:
OPCON is the command authority that may be exercised by commanders at any
echelon at or below the level of CCMD and may be delegated within the command.


Ask a Question from the PDF Files (type 'exit' to quit): what is TACON
{'output_text': 'TACON is an authority over assigned or attached forces or commands, or military capability or forces made available for tasking, that is limited to the detailed direction and control of movements and maneuvers within the operational area necessary to accomplish assigned missions or tasks assigned by the commander exercising OPCON or TACON of the attached force.'}

Question:
what is TACON

Answer:
TACON is an authority over assigned or attached forces or commands, or military
capability or forces made available for tasking, that is limited to the